In [120]:
# Adding the Aima-Python folder to the folder

import sys
import random
from random import choice
sys.path.insert(1, 'D:/Masters/Knowledge Representation/workplace/aima-python')

In [51]:
from agents import *
from search import *

##### Creation of things in the environment

In [52]:
class LifeJacket(Thing):
    # This will provide the life jacket to the Player and will remove all enery down points    
    pass

class PowerUp(Thing):
    # Increases the performance of the agent by + 1
    pass

class PowerDown(Thing):
     # Decreases the performance of the agent by - 1
    pass

class EnergyUp(Thing):
     # Increases the performance of the agent by + 4
    pass

class EnergyDown(Thing):
     # Decreases the performance of the agent by - 4
    pass

class WinningRow(Thing):
    # Perfromance change in case of winning row
    pass

class Barrier(Thing):
    # This will not allow the Player to move ahead, and force to find alternative way
    pass

class PitFall(Thing):
    # This will kill the Player
    pass


#### Defining the Program method
The program method works on the percepts and return the appropriate action to be used by execute_action

In [82]:
def program(percepts):
    '''Returns an action based on it's percepts'''
    
    #print("Dheeraj" , percepts) # Contains list of things that are curently faced by agent in environment for tabledriven
    
    for values_percepted in percepts: 
        if values_percepted == 'goalsearch':
            return 'searchproblem'        
        
        if isinstance(values_percepted, PowerUp):
            return 'powerup'
        
        elif isinstance(values_percepted, PowerDown):
            return 'powerdown'
        
        elif isinstance(values_percepted, EnergyUp):
            return 'energyup'

        elif isinstance(values_percepted, EnergyDown):
            return 'energydown'
        
        elif isinstance(values_percepted, PitFall):
            return 'pitfall'
        
        elif isinstance(values_percepted, LifeJacket):
            return 'grab'
        
        if isinstance(values_percepted, Barrier):
            return 'barrier'
        
        if isinstance(values_percepted,WinningRow):
            return 'lastlevel'
            
        if isinstance(values_percepted,Bump):
            # then check if you are at an edge and have to turn
            turn = False
            choice = random.choice((1,2));
            
        else:
            choice = random.choice((1,2,3,4,5,6,7)) # 1-right, 2-left, others-forward

    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'

#### This creates the 2d Environment which contains the percepts, execute_action  and the is_done method

In [437]:
class RaceFieldEnvironment(GraphicEnvironment):
    
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        #print(self.things_near(agent.location,2))
        
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        
        if agent.goalsearch == True:
            goalsearch = ['goalsearch']
            return goalsearch
        
        if agent.location[1] == 0 :
            things.append(WinningRow())  # Adding the thing to know if the target row is reached
            
        elif not self.is_inbounds(loc):
            things.append(Bump())

        return things
    
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)   
            
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
            
        elif action == 'moveforward':
            if isinstance(agent,PlayerA):
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
            
            elif isinstance(agent,PlayerB):
                """
                The Agent will see all the things near to him in the environment.
                If there is an Energy Down in the next near cell(not Adjacent), it will change the path
                """
                near_things = self.things_near(agent.location,2)
  
                if 5 < agent.lives <= agent.max_lives and any(isinstance(element[0],PitFall) and element[1] == 3  for element in near_things):
                    """
                    The Agent will see all the things near to him in the environment.
                    If there is an Pitfall in the next near cell(not Adjacent), it will Jump the pitfall
                    """
                    for element in near_things:
                        if isinstance(element[0],PitFall):
                            if element[1] == 3:   # This will check if only the near cells(not adjacent) have energy drop
                                current_location = agent.location
                                forward_check = [current_location[0],current_location[1]-1]
                                backward_check = [current_location[0],current_location[1]+1]
                                left_check = [current_location[0]-1,current_location[1]]
                                right_check = [current_location[0]+1,current_location[1]]

                                # The following conditions check in which cell the energy down is and change direction accordingly
                                if any(isinstance(element,PitFall) for element in self.list_things_at(forward_check)):                                                          
                                    print('Found Pitfall ahead, {} decided to Jump at location: {}'.format(str(agent)[1:-1] , agent.location))
                                    agent.location = forward_check
                                    agent.direction.direction = Direction.U
                                    agent.lives = 5
                                
                                elif any(isinstance(element,PitFall) for element in self.list_things_at(left_check)):
                                    print('Found Pitfall Leftwards, {} decided to Jump at location: {}'.format(str(agent)[1:-1] , agent.location))
                                    agent.location = left_check
                                    agent.direction.direction = Direction.L
                                    agent.lives = 5
                                    
                                elif any(isinstance(element,PitFall) for element in self.list_things_at(right_check)):
                                    print('Found Pitfall Rightwards, {} decided to Jump at location: {}'.format(str(agent)[1:-1] , agent.location))                                    
                                    agent.location = right_check
                                    agent.direction.direction = Direction.R
                                    agent.lives = 5

                                break #In case the energy down is found exit the loop         

                elif agent.lives < agent.max_lives and any(isinstance(element[0],EnergyDown) and element[1] == 3  for element in near_things):
                    # This will check if the near element has energy down
                    # if any(isinstance(element[0],EnergyDown) and element[1] == 3  for element in near_things) :

                    for element in near_things:
                        if isinstance(element[0],EnergyDown):
                            if element[1] == 3:   # This will check if only the near cells(not adjacent) have energy drop
                                current_location = agent.location
                                forward_check = [current_location[0],current_location[1]-1]
                                backward_check = [current_location[0],current_location[1]+1]
                                left_check = [current_location[0]-1,current_location[1]]
                                right_check = [current_location[0]+1,current_location[1]]

                                # The following conditions check in which cell the energy down is and change direction accordingly
                                if any(isinstance(element,EnergyDown) for element in self.list_things_at(forward_check)):
                                    agent.direction.direction = Direction.R
                                    point = 'ahead'
                                    #print('Found Energy Drop ahead, {} decided to move {}wards at location: {}'.format(str(agent)[1:-1] ,agent.direction.direction, agent.location))

                                elif any(isinstance(element,EnergyDown) for element in self.list_things_at(backward_check)):

                                    agent.direction.direction = Direction.U
                                    point = 'behind'
                                    #print('Found Energy Drop behind, {} decided to move {}wards at location: {}'.format(str(agent)[1:-1] ,agent.direction.direction, agent.location))

                                elif any(isinstance(element,EnergyDown) for element in self.list_things_at(left_check)):

                                    agent.direction.direction = Direction.U
                                    point = 'Left'
                                    #print('Found Energy Drop on Left Side, {} decided to move {}wards at location: {}'.format(str(agent)[1:-1] ,agent.direction.direction, agent.location))                           

                                elif any(isinstance(element,EnergyDown) for element in self.list_things_at(right_check)):

                                    agent.direction.direction = Direction.U
                                    point = 'Right'
                                    #print('Found Energy Drop on Right Side, {} decided to move {}wards at location: {}'.format(str(agent)[1:-1] ,agent.direction.direction, agent.location))
                                
                                print('Found Energy Drop {}, {} decided to move {}wards at location: {}'.format(point,str(agent)[1:-1] ,agent.direction.direction, agent.location))
                                break #In case the energy down is found exit the loop
                
                else:
                    # In case No nearby cell has enegy down, continue normally
                    # In case No nearby cell has a pitfall, continue normally
                    print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))

                agent.moveforward()
                            
        elif action == 'lastlevel' :
            agent.direction.direction = Direction.L
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1] ,agent.direction.direction, agent.location))
            agent.moveforward()
            
        elif action == "powerup":
            items = self.list_things_at(agent.location, tclass=PowerUp)
            if len(items) != 0:
                if agent.powerup(items[0]):

                    if isinstance(agent,PlayerA):
                        agent.lives +=1
                        print('{} received a power up by having {} at location: {}'
                              .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                        self.delete_thing(items[0])
                    
                    elif isinstance(agent,PlayerB):

                        if agent.lives == agent.max_lives:
                            print('{} ignores a power up at location: {} as its Power is Full'
                                  .format(str(agent)[1:-1], agent.location))
                            agent.direction.direction = Direction.U
                            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                            agent.moveforward()
                                                      
                        else:
                            agent.lives +=1
                            print('{} received a power up by having {} at location: {}'
                                  .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                            self.delete_thing(items[0])            
                                           
        elif action == "powerdown":
            items = self.list_things_at(agent.location, tclass=PowerDown)
            if len(items) != 0:
                if agent.powerdown(items[0]):
                    print('{} received a power down by having {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
        elif action == "energyup":            
            items = self.list_things_at(agent.location, tclass=EnergyUp)
            if len(items) != 0:
                if agent.energyup(items[0]):
                    if isinstance(agent,PlayerA):
                        agent.lives += 4
                        print('{} energized by having {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                        self.delete_thing(items[0])
                        
                    elif isinstance(agent,PlayerB):                        
                        if agent.lives == agent.max_lives:
                            print('{} ignores a energy up at location: {} as its Power is Full'
                                  .format(str(agent)[1:-1], agent.location))
                            agent.direction.direction = Direction.U
                            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                            agent.moveforward()
                                                        
                        else:                            
                            agent.lives += 4
                            if agent.lives > agent.max_lives:
                                agent.lives = 10 # This is the maximum allowed power of the player
                            print('{} energized by having {} at location: {}'
                                  .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                            self.delete_thing(items[0])                                    
                        
        elif action == "energydown":
            items = self.list_things_at(agent.location, tclass=EnergyDown)
            if len(items) != 0:
                if agent.energydown(items[0]):
                    print('{} drained by having {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        
        elif action == 'grab':
            things = [thing for thing in self.list_things_at(agent.location)
                      if agent.can_grab(thing)]
            if len(things):
                print('{} grabbed {} at location: {}'
                          .format(str(agent)[1:-1],things[0].__class__.__name__, agent.location))
                print('Removing all Energy Down Points')
                #print("Grabbing", things[0].__class__.__name__)
                if len(things):
                    agent.holding.append(things[0])                   
                    self.delete_thing(things[0])                    
                    for element in self.things:
                        # Updating the list of things
                        self.things[:] = (element for element in self.things if not isinstance(element,EnergyDown))
                                        
        elif action == "barrier":            
            items = self.list_things_at(agent.location, tclass=Barrier)
            if len(items) !=0:
                if agent.barrier(items[0]):
                    agent.direction.direction = Direction.L
                    print('{} encountered barrier and decided to move {}wards at location: {}'.format(str(agent)[1:-1] ,agent.direction.direction, agent.location))
                    agent.moveforward()                   
                    
        elif action == "pitfall":
            items = self.list_things_at(agent.location, tclass=PitFall)
            if len(items) != 0:
                if agent.pitfall(items[0]):
                    print('{} died by falling into {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])       
                    agent.alive = False
        
        elif action == "searchproblem":   
            agent.display_performance = True
            agent.location = agent.goalpath[0]
            print('{} decided to move at location: {}'.format(str(agent)[1:-1], agent.location))
            agent.steps +=1
            agent.goalpath.pop(0)

                           
    def is_done(self):  
        
        if self.agents[0].display_performance:
            print(f'Current Lives with {str(self.agents[0])[1:-1]} : {self.agents[0].lives}')
            self.agents[0].display_performance = False
        
        if self.agents[0].location == [0,0]:  # This is for the Player A if he reaches the Goal
            print(f'{str(self.agents[0])[1:-1]} won the game in {self.agents[0].steps} steps')
            self.agents[0].alive = False
            return True
        
        elif self.agents[0].lives <= 0:
            print(f'{str(self.agents[0])[1:-1]} died before reaching Goal with 0 lives')
            print('GAME OVER')
            self.agents[0].alive = False
            return True           

        dead_agents = not any(agent.is_alive() for agent in self.agents)
        #return dead_agents or no_edibles
        return dead_agents

#### Setting up the environment with agents and things

In [164]:
def agent_run(agent_name,steps,player = None, seed = False):
    
    racefield = RaceFieldEnvironment(6,6, color={agent_name: (25, 32, 33),
                                             'PowerUp': (157, 200, 0), 'PowerDown': (200, 103, 0),
                                             'EnergyUp': (77, 200, 0), 'EnergyDown': (200, 0, 0),
                                             'PitFall':(214, 30, 107), 'Barrier': (240, 222, 29),
                                             'LifeJacket':(66, 135, 245)
                                            })
    # Creating the objects for the things
    powerup = PowerUp()
    powerdown = PowerDown()
    energyup = EnergyUp()
    energydown = EnergyDown()
    energydown_1 = EnergyDown()
    energydown_2 = EnergyDown()
    barrier = Barrier()
    lifejacket = LifeJacket()
    pitfall = PitFall()
    
    # Choosing a random start location
    if seed:
        start_location = seed
    
    else:
        start_location = random.choice([[1,5],[2,5],[3,5],[4,5],[5,5]])
        
    racefield.add_thing(player, start_location) # Adding player A on the centre of last row

    # TraceAgent(playerA)
    _dict_things = {powerup:[0,5],powerdown: [2,1],energyup: [4,2],
                    energydown: [0,3],energydown_1: [4,4],energydown_2: [2,2],
                    barrier:[5,2], lifejacket:[2,4], pitfall:[1,1]
                   }

    # Adding the things on location
    for thing,loc in _dict_things.items():
        racefield.add_thing(thing,loc)

    #print(racefield.get_world())  # Shows the current world

    print(f'{player.__class__.__name__}  started at {start_location}, facing up. See if he Wins')
    racefield.run(steps)

    if not player.holding:
        print(f'The {player.__class__.__name__}  did not grab anything in the Game')
    else:
        print(f'{player.__class__.__name__} grabbed the following in Game:')
        for grabs in player.holding:
            print(str(grabs)[1:-1])

    if player.is_alive():
        print('===================================================\n')
        print(f'{player.__class__.__name__} Exhausted all the steps before reaching goal(max. allowed steps= {steps})\n')
        print(f' Goal: [0,0], current location {player.location} \n')
        print('===================================================')


## Agent 1 Simple Reflex Agent

This agent will know what is the immediate environment and will take the action accordingly

1. If the agent see's that there is a energy down in any adjacent cell, it will avoid going there
2. If agent's Current Live is 10(Maximum), then agent will avoid going to Power Up as it cannot hold more power
3. If a agent has Life Jacket in any adjacent cell, it will grab it.
4. If the agent see a pitfall, it will Jump over it.

#### Creating the Agent (Player A)
Agent :Player A is a Simple Reflex Agent
1. It percepts upon the items present in the environment
2. It percept the Wall and takes appropriate change in direction
3. It percepts the Winning Row and take the appropriate direction of moving towards goal
4. It percepts the available Power Ups and increase it Life COunt
5. It percepts the encounter of the Power Down and decreases its Life Count
6. It percepts the encounter of the Energy Up and increases its Life Count
7. It percepts the encounter of the Energy Down and decreases its Life Count
8. It percepts the Pitfall and dies 

In [112]:
class PlayerA(Agent):
    
    """This agent takes action based solely on the percept. <SIMPLE REFLEX AGENT>"""
    
    def __init__(self, program=None, goalpath = None, goalsearch=False):
        Agent.__init__(self,program)
        self.goalsearch = goalsearch
        self.goalpath = goalpath
    
    location = [0,3]
    direction = Direction("up")
    steps = 0
    max_lives = 10
    lives = max_lives   # lives with player on game start
    display_performance = True

    def can_grab(self, thing):
        """Explorer can only grab Life Jacket"""
        return thing.__class__ == LifeJacket    

    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        self.display_performance = True
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
            self.steps +=1 
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
            self.steps +=1 
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
            self.steps +=1 
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
            self.steps +=1 

    def turn(self, d):
        self.display_performance = True
        self.direction = self.direction + d

        
    def powerup(self, thing):
        '''returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, PowerUp):
            #self.lives +=1
            return True
        return False
    
    def powerdown(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, PowerDown):
            self.lives -=1
            return True
        return False
    
    def energyup(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, EnergyUp):
            #self.lives += 4
            return True
        return False

    def energydown(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, EnergyDown):
            self.lives -= 4
            return True
        return False
    
    def pitfall(self,thing):
        if isinstance(thing,PitFall):
            return True
        return False
        
    def barrier(self,thing):
        if isinstance(thing,Barrier):
            return True
        return False
        
           

In [430]:
agent_run('PlayerA',100,PlayerA(program,goalsearch=False))

PlayerA  started at [1, 5], facing up. See if he Wins


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA decided to move leftwards at location: [1, 5]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA received a power up by having PowerUp at location: [0, 5]


...

,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 7
PlayerA died by falling into PitFall at location: [1, 1]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


The PlayerA  did not grab anything in the Game


## Agent 2 Model Based Reflex Agent

This agent will know what is the immediate environment and will take the action accordingly

1. If the agent see's that there is a energy down in any adjacent cell, it will avoid going there
2. If agent's Current Live is 10(Maximum), then agent will avoid going to Power Up as it cannot hold more power
3. If a agent has Life Jacket in any adjacent cell, it will grab it.
4. If the agent see a pitfall, it will Jump over it.

#### Creating the Agent (Player B)

Agent :Player B is a Model Based Reflex Agent
1. It percepts upon the items present in the environment <b>Done</b>

2. It percept the Wall and takes appropriate change in direction <b>Done</b>

3. It percepts the Winning Row and take the appropriate direction of moving towards goal <b>Done</b>

4. It percepts the available Power Ups and increase it Life Count <b>Done</b>
    <br>a. If agent's Current Live is 10(Maximum), then agent will avoid going to Power Up as it cannot hold more power - <b>Done</b>
    
5. It percepts the encounter of the Power Down and decreases its Life Count <b>Done</b>

6. It percepts the encounter of the Energy Up and increases its Life Count <b>Done</b>
    <br>a. If agent's Current Live is 10(Maximum), then agent will avoid going to Energy Up as it cannot hold more power <b>Done</b>
    <br>b. If after taking the Energy Up if the energy goes > 10 , it should keep it maximum to 10 <b>Done</b>
    
7. It percepts the encounter of the Energy Down and decreases its Life Count
    <br>a. If the agent live is < max lives, then its see's that there is a energy down in any near(non adjacent) cell, it will avoid going there -<b>Done</b>

8. It percepts the Pitfall and dies <b>Done</b>
    <br>a. If the agent see a pitfall, it will Jump over it if the lives are between 5 and 10. This operation has a cost,which reduces the agent lives to 4

In [10]:
class PlayerB(Agent):
    
    """This agent takes action based on the percept and the near environment. <MODEL BASED REFLEX AGENT>"""
    
    location = [0,3]
    direction = Direction("up")
    steps = 0
    max_lives = 10
    lives = max_lives   # lives with player on game start
    display_performance = True

    def can_grab(self, thing):
        """Explorer can only grab Life Jacket"""
        return thing.__class__ == LifeJacket    

    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        self.display_performance = True
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
            self.steps +=1 
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
            self.steps +=1 
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
            self.steps +=1 
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
            self.steps +=1 

    def turn(self, d):
        self.display_performance = True
        self.direction = self.direction + d

        
    def powerup(self, thing):
        '''returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, PowerUp):
            #self.lives +=1
            return True
        return False
    
    def powerdown(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, PowerDown):
            self.lives -=1
            return True
        return False
    
    def energyup(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, EnergyUp):
            #self.lives += 4
            return True
        return False

    def energydown(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, EnergyDown):
            self.lives -= 4
            return True
        return False
    
    def pitfall(self,thing):
        if isinstance(thing,PitFall):
            return True
        return False
        
    def barrier(self,thing):
        if isinstance(thing,Barrier):
            return True
        return False
        
           

In [16]:
agent_run('PlayerB',100,PlayerB(program))

PlayerB  started at [1, 5], facing up. See if he Wins


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerB : 10
PlayerB decided to move leftwards at location: [1, 5]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerB : 10
PlayerB ignores a power up at location: [0, 5] as its Power is Full
PlayerB decided to move upwards at location: [0, 5]


...

Current Lives with PlayerB : 5
PlayerB decided to move leftwards at location: [1, 0]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerB : 5
PlayerB won the game in 7 steps


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


The PlayerB  did not grab anything in the Game


## SEARCH

#### Defining the Problem Class which will be the base class for the Problem Statement

In [141]:
import math , sys
from collections import deque

In [142]:
class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        
        raise NotImplementedError

    def result(self, state, action): 
        raise NotImplementedError

    def is_goal(self, state):        
        return state == self.goal

    def action_cost(self, s, a, s1): 
        return 1

    def h(self, node):               
        return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(type(self).__name__, self.initial, self.goal)

#### Defining the Node Class ( A node in the search tree)

In [143]:
class Node:
    
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): 
        return '<{}>'.format(self.state)

    def __len__(self): 
        return 0 if self.parent is None else (1 + len(self.parent))

    def __lt__(self, other): 
        return self.path_cost < other.path_cost


    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[:]]

    
failure = Node('failure', path_cost=math.inf)  # Indicates an algorithm couldn't find a solution.
cutoff = Node('cutoff', path_cost=math.inf)  # Indicates iterative deepening search was cut off.

In [144]:
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)

In [145]:
FIFOQueue = deque

LIFOQueue = list

### The Search Technique : Breadth First Search

In [146]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

### The Search Technique : Depth Limited Search

In [454]:
def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"

    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))

    return find_cycle(node.parent, k)

def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result

### The Search Technique : Iterative Deepening Search

In [450]:
def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result

### Defining the Problem Statement class 

In [149]:
class GoalSearch(Problem):
    
    """Finding a path on a 2D grid with obstacles. Obstacles are (x, y) cells."""

    def __init__(self, initial=(5,5), goal=(0,0), obstacles=(), **kwds):
        
        Problem.__init__(self, initial=initial, goal=goal,obstacles=set(obstacles) - {initial, goal}, **kwds)

    directions = [(0, -1),(-1, 0), (+1, 0),(0, +1)]
    
    def action_cost(self, s, action, s1):

        return straight_line_distance(s, s1)

    def h(self, node):

        return straight_line_distance(node.state, self.goal)

    def result(self, state, action):
        "Both states and actions are represented by (x, y) pairs."
        return action if action not in self.obstacles else state

    def actions(self, state):
        """You can move one cell in any of `directions` to a non-obstacle cell."""
        x, y = state

        return {(x + dx, y + dy) for (dx, dy) in self.directions} - self.obstacles
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [459]:
seed_randomizer = [34,5,656,578,98,35,52,3,5,67,78,797,99,898,895586,63626,26,986,9,869,68,96]
x = random.choice(seed_randomizer)
random.seed(x)
start_location = random.choice([[1,5],[2,5],[3,5],[4,5],[5,5]])
start_location_search = tuple(start_location)
searchproblem = GoalSearch(initial = start_location_search,obstacles={(4,4),(0,3),(2,2),(5,2),(2,1),(1,1)})

##### Goal Test with Breadth First Search

In [441]:
print('Breadth First Search \n Shotest Route :', breadth_first_search(searchproblem).solution())
goalpath_elements = breadth_first_search(searchproblem).solution()
goalpath = []
for element in goalpath_elements:
    goalpath.append(list(element))
    
agent_run('PlayerA',100,PlayerA(program,goalpath, goalsearch=True),start_location)

Breadth First Search 
 Shotest Route : [(1, 5), (1, 4), (1, 3), (1, 2), (0, 2), (0, 1), (0, 0)]
PlayerA  started at [2, 5], facing up. See if he Wins


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA decided to move at location: [1, 5]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA decided to move at location: [1, 4]


...

Current Lives with PlayerA : 10
PlayerA decided to move at location: [0, 0]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA won the game in 7 steps


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


The PlayerA  did not grab anything in the Game


##### Goal Test with Depth Limited Search

In [455]:
print('Depth Limited Search \n Shortest Path = ', depth_limited_search(searchproblem).solution())

goalpath_elements = depth_limited_search(searchproblem).solution()
goalpath = []
for element in goalpath_elements:
    goalpath.append(list(element))
    
agent_run('PlayerA',100,PlayerA(program,goalpath, goalsearch=True),start_location)

Depth Limited Search 
 Shortest Path =  [(3, 5), (3, 4), (3, 3), (2, 3), (1, 3), (1, 2), (0, 2), (0, 1), (0, 0)]
PlayerA  started at [2, 5], facing up. See if he Wins


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA decided to move at location: [3, 5]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA decided to move at location: [3, 4]


...

Current Lives with PlayerA : 10
PlayerA decided to move at location: [0, 0]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA won the game in 9 steps


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


The PlayerA  did not grab anything in the Game


##### Goal Test with Iterative Deepening Search

In [460]:
print('Iterative Deepening Search \n Shortest Path = ', iterative_deepening_search(searchproblem).solution())

goalpath_elements = iterative_deepening_search(searchproblem).solution()
goalpath = []
for element in goalpath_elements:
    goalpath.append(list(element))
    
agent_run('PlayerA',100,PlayerA(program,goalpath, goalsearch=True),start_location)

Iterative Deepening Search 
 Shortest Path =  [(3, 4), (3, 3), (2, 3), (1, 3), (1, 2), (0, 2), (0, 1), (0, 0)]
PlayerA  started at [3, 5], facing up. See if he Wins


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA decided to move at location: [3, 4]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA decided to move at location: [3, 3]


...

Current Lives with PlayerA : 10
PlayerA decided to move at location: [0, 0]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 10
PlayerA won the game in 8 steps


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


The PlayerA  did not grab anything in the Game
